Extract and arrange the dataset

In [ ]:
import os
import zipfile
import shutil
import re

zip_file = "Faces.zip"  # Ensure this is uploaded to your Colab environment.
extracted_folder = "Faces_unorganized"
structured_folder = "Faces"

# Step 1: Extract the zip file
print("Extracting zip file...")
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)
print("Extraction completed. Files extracted into:", extracted_folder)

# Step 2: Arrange images into structured folders.
print("Arranging files into structured folders...")
os.makedirs(structured_folder, exist_ok=True)

# Recursively walk through the extracted folder
for root, dirs, files in os.walk(extracted_folder):
    for filename in files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Extract label: everything before the underscore followed by digits.
            match = re.match(r'^(.*?)_\d+', filename)
            if match:
                label = match.group(1)
            else:
                label = os.path.splitext(filename)[0]

            # Create a subfolder for this label if it doesn't exist.
            label_dir = os.path.join(structured_folder, label)
            os.makedirs(label_dir, exist_ok=True)

            # Define source and destination paths.
            src = os.path.join(root, filename)
            dst = os.path.join(label_dir, filename)

            #print(f"Moving file {src} to {dst}")
            shutil.move(src, dst)

print("Dataset arranged in folder:", structured_folder)


Extracting zip file...
Extraction completed. Files extracted into: Faces_unorganized
Arranging files into structured folders...
Dataset arranged in folder: Faces


Define data transformations. Load the dataset from the arranged folder.Split the dataset into training and validation sets. Create DataLoaders for both sets.

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define transformations: resize images, convert to tensor, and normalize.
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),      # Resize all images to 128x128 pixels.
    transforms.ToTensor(),              # Convert images to PyTorch tensors.
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize for 3-channel images.
])

# Load the dataset from the arranged folder "Faces".
data_dir = "Faces"  # Make sure this folder is in your Colab working directory.
dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)
print("Classes found:", dataset.classes)

# Split the dataset into training and validation sets (80-20 split).
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

# Create DataLoaders for training and validation.
batch_size = 32  # Adjust the batch size if needed.
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Display one batch of images and labels for verification.
dataiter = iter(train_loader)
images, labels = next(dataiter)
print("Batch image tensor shape:", images.shape)
print("Batch labels:", labels)

Classes found: ['Akshay Kumar', 'Alexandra Daddario', 'Alia Bhatt', 'Amitabh Bachchan', 'Andy Samberg', 'Anushka Sharma', 'Billie Eilish', 'Brad Pitt', 'Camila Cabello', 'Charlize Theron', 'Claire Holt', 'Courtney Cox', 'Dwayne Johnson', 'Elizabeth Olsen', 'Ellen Degeneres', 'Henry Cavill', 'Hrithik Roshan', 'Hugh Jackman', 'Jessica Alba', 'Kashyap', 'Lisa Kudrow', 'Margot Robbie', 'Marmik', 'Natalie Portman', 'Priyanka Chopra', 'Robert Downey Jr', 'Roger Federer', 'Tom Cruise', 'Vijay Deverakonda', 'Virat Kohli', 'Zac Efron']
Training samples: 2049, Validation samples: 513
Batch image tensor shape: torch.Size([32, 3, 128, 128])
Batch labels: tensor([18, 30,  7, 17,  8,  7, 18, 23, 12, 24, 27, 18,  7, 27, 30,  6,  4, 15,
        28, 12, 29,  9,  4, 17, 15, 27, 24,  3, 15, 18, 13, 24])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FaceCNN(nn.Module):
    def __init__(self, num_classes, input_channels=3, input_height=128, input_width=128):
        super(FaceCNN, self).__init__()
        self.num_pool_layers = 4  # We will use 4 pooling operations (each halves H & W)

        # Convolutional Block 1
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        # Convolutional Block 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        # Convolutional Block 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        # Convolutional Block 4
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        self.pool = nn.MaxPool2d(2, 2)

        # Compute the size of the feature map after the pooling layers
        final_height = input_height // (2 ** self.num_pool_layers)
        final_width = input_width // (2 ** self.num_pool_layers)
        fc_input_dim = 256 * final_height * final_width

        # Fully Connected Layers
        self.fc1 = nn.Linear(fc_input_dim, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Convolutional Blocks with pooling
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        # Flatten feature maps
        x = x.view(x.size(0), -1)
        # Fully Connected Layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Example of how to instantiate the model dynamically:
# Get a sample batch from your train_loader to extract input dimensions.
sample_images, _ = next(iter(train_loader))
input_channels = sample_images.shape[1]
input_height = sample_images.shape[2]
input_width = sample_images.shape[3]

# Dynamically determine the number of classes from your dataset.
num_classes = len(dataset.classes)

# Create the model instance with dynamic parameters.
model = FaceCNN(num_classes=num_classes, input_channels=input_channels,
                input_height=input_height, input_width=input_width)

# Move the model to GPU if available.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

#print(model)

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Make sure that the following variables are defined:
# model, train_loader, val_loader, train_dataset, val_dataset, device, num_classes
# Also define your loss function and optimizer, for example:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Use a learning rate scheduler to reduce the learning rate when validation loss plateaus.
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

best_val_loss = float('inf')
num_epochs = 10  # Adjust as needed

# Early stopping parameters
early_stop_patience = 5
epochs_no_improve = 0

# Logging history for later analysis (optional)
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    epoch_start_time = time.time()

    # ----- Training Phase -----
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training")
    for images, labels in train_bar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()

        # Clip gradients to help with stability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)

        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        train_bar.set_postfix(loss=loss.item())

    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy = 100 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    print(f"\nEpoch [{epoch+1}/{num_epochs}] Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # ----- Validation Phase -----
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation")
    with torch.no_grad():
        for images, labels in val_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            val_bar.set_postfix(loss=loss.item())

    val_loss = val_loss / len(val_dataset)
    val_accuracy = 100 * correct_val / total_val
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}] Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")

    # Step the scheduler using validation loss
    scheduler.step(val_loss)

    # Log current learning rate
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Current Learning Rate: {current_lr}")

    # Save the best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print("Model checkpoint saved!")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    # Early stopping check
    if epochs_no_improve >= early_stop_patience:
        print("Early stopping triggered.")
        break

    epoch_duration = time.time() - epoch_start_time
    print(f"Epoch duration: {epoch_duration:.2f} seconds\n")


Epoch 1/10 - Training: 100%|██████████| 65/65 [01:47<00:00,  1.65s/it, loss=3.91]



Epoch [1/10] Training Loss: 2.4775, Accuracy: 71.74%


Epoch 1/10 - Validation: 100%|██████████| 17/17 [00:16<00:00,  1.04it/s, loss=0.708]


Epoch [1/10] Validation Loss: 0.7561, Accuracy: 88.11%
Current Learning Rate: 5e-05
Model checkpoint saved!
Epoch duration: 123.96 seconds



Epoch 2/10 - Training: 100%|██████████| 65/65 [01:46<00:00,  1.64s/it, loss=3.94]


...

Epoch [9/10] Validation Loss: 0.3043, Accuracy: 92.40%
Current Learning Rate: 5e-05
Model checkpoint saved!
Epoch duration: 122.77 seconds



Epoch 10/10 - Training: 100%|██████████| 65/65 [01:47<00:00,  1.66s/it, loss=3.74]



Epoch [10/10] Training Loss: 0.0057, Accuracy: 99.95%


Epoch 10/10 - Validation: 100%|██████████| 17/17 [00:16<00:00,  1.05it/s, loss=0.149]

Epoch [10/10] Validation Loss: 0.3434, Accuracy: 92.20%
Current Learning Rate: 5e-05
Epoch duration: 124.07 seconds



In [ ]:
# import time
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import models, transforms
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# from tqdm import tqdm

# #########################################
# # Enhanced Data Augmentation (Milder Version)
# #########################################
# data_transforms = transforms.Compose([
#     transforms.Resize((128, 128)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(20),  # Reduced rotation range
#     transforms.RandomCrop(128, padding=5),  # Reduced padding
#     transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
# ])

# #########################################
# # Define ArcMarginProduct with Adjusted Hyperparameters
# #########################################
# class ArcMarginProduct(nn.Module):
#     def __init__(self, in_features, out_features, s=30.0, m=0.35, easy_margin=False):
#         super(ArcMarginProduct, self).__init__()
#         self.in_features = in_features
#         self.out_features = out_features
#         self.s = s  # scale factor
#         self.m = m  # angular margin (reduced from 0.50 to 0.35)
#         self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
#         nn.init.xavier_uniform_(self.weight)
#         self.easy_margin = easy_margin
#         self.cos_m = torch.cos(torch.tensor(m))
#         self.sin_m = torch.sin(torch.tensor(m))
#         self.th = torch.cos(torch.tensor(torch.pi) - m)
#         self.mm = torch.sin(torch.tensor(torch.pi) - m) * m

#     def forward(self, input, label):
#         cosine = F.linear(F.normalize(input), F.normalize(self.weight))
#         sine = torch.sqrt(1.0 - torch.pow(cosine, 2) + 1e-6)
#         phi = cosine * self.cos_m - sine * self.sin_m
#         if self.easy_margin:
#             phi = torch.where(cosine > 0, phi, cosine)
#         else:
#             phi = torch.where(cosine > self.th, phi, cosine - self.mm)
#         one_hot = torch.zeros(cosine.size(), device=input.device)
#         one_hot.scatter_(1, label.view(-1, 1).long(), 1)
#         output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
#         output *= self.s
#         return output

# #########################################
# # Transfer Learning: Pretrained ResNet18 with Fine-Tuning
# #########################################
# num_classes = 31  # Adjust based on your dataset
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load a pretrained ResNet18 model (using weights= parameter to avoid deprecation warnings)
# pretrained_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# # Freeze all layers first
# for param in pretrained_model.parameters():
#     param.requires_grad = False

# # Unfreeze the last residual block (layer4) for fine-tuning
# for param in pretrained_model.layer4.parameters():
#     param.requires_grad = True

# # Replace the final fully connected layer to output a 512-dimensional feature vector
# in_features = pretrained_model.fc.in_features
# pretrained_model.fc = nn.Linear(in_features, 512)
# # Ensure the new fc layer is trainable
# for param in pretrained_model.fc.parameters():
#     param.requires_grad = True

# #########################################
# # Combine Backbone with Adjusted ArcFace Layer
# #########################################
# class FaceRecognitionModel(nn.Module):
#     def __init__(self, backbone, feature_dim, num_classes, s=30.0, m=0.35):
#         super(FaceRecognitionModel, self).__init__()
#         self.backbone = backbone
#         self.arc_margin = ArcMarginProduct(feature_dim, num_classes, s=s, m=m, easy_margin=False)

#     def forward(self, x, label=None):
#         features = self.backbone(x)  # shape: [batch, feature_dim]
#         if label is not None:
#             output = self.arc_margin(features, label)
#         else:
#             output = features
#         return output

# model = FaceRecognitionModel(pretrained_model, feature_dim=512, num_classes=num_classes, s=30.0, m=0.35)
# model = model.to(device)

# #########################################
# # Define Loss, Optimizer, and Scheduler
# #########################################
# criterion = nn.CrossEntropyLoss()
# # Use a lower learning rate for fine-tuning
# optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001, weight_decay=1e-5)
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# #########################################
# # Training Loop with Extended Patience
# #########################################
# best_val_loss = float('inf')
# num_epochs = 30  # Increase number of epochs for fine-tuning
# early_stop_patience = 10  # Increase patience for early stopping
# epochs_no_improve = 0

# train_losses = []
# val_losses = []
# train_accuracies = []
# val_accuracies = []

# for epoch in range(num_epochs):
#     epoch_start_time = time.time()

#     # ----- Training Phase -----
#     model.train()
#     running_loss = 0.0
#     correct = 0
#     total = 0

#     train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training")
#     for images, labels in train_bar:
#         images, labels = images.to(device), labels.to(device)
#         optimizer.zero_grad()

#         outputs = model(images, labels)
#         loss = criterion(outputs, labels)
#         loss.backward()

#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
#         optimizer.step()

#         running_loss += loss.item() * images.size(0)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#         train_bar.set_postfix(loss=loss.item())

#     epoch_loss = running_loss / len(train_dataset)
#     epoch_accuracy = 100 * correct / total
#     train_losses.append(epoch_loss)
#     train_accuracies.append(epoch_accuracy)

#     print(f"\nEpoch [{epoch+1}/{num_epochs}] Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

#     # ----- Validation Phase -----
#     model.eval()
#     val_loss = 0.0
#     correct_val = 0
#     total_val = 0

#     val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation")
#     with torch.no_grad():
#         for images, labels in val_bar:
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images, labels)
#             loss = criterion(outputs, labels)
#             val_loss += loss.item() * images.size(0)

#             _, predicted = torch.max(outputs.data, 1)
#             total_val += labels.size(0)
#             correct_val += (predicted == labels).sum().item()
#             val_bar.set_postfix(loss=loss.item())

#     val_loss = val_loss / len(val_dataset)
#     val_accuracy = 100 * correct_val / total_val
#     val_losses.append(val_loss)
#     val_accuracies.append(val_accuracy)

#     print(f"Epoch [{epoch+1}/{num_epochs}] Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")

#     scheduler.step(val_loss)
#     current_lr = optimizer.param_groups[0]['lr']
#     print(f"Current Learning Rate: {current_lr}")

#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         torch.save(model.state_dict(), 'best_model.pth')
#         print("Model checkpoint saved!")
#         epochs_no_improve = 0
#     else:
#         epochs_no_improve += 1

#     if epochs_no_improve >= early_stop_patience:
#         print("Early stopping triggered.")
#         break

#     epoch_duration = time.time() - epoch_start_time
#     print(f"Epoch duration: {epoch_duration:.2f} seconds\n")


Epoch 1/30 - Training: 100%|██████████| 65/65 [01:47<00:00,  1.65s/it, loss=15.7]



Epoch [1/30] Training Loss: 11.6306, Accuracy: 0.39%


Epoch 1/30 - Validation: 100%|██████████| 17/17 [00:16<00:00,  1.01it/s, loss=11.5]


Epoch [1/30] Validation Loss: 8.9091, Accuracy: 6.04%
Current Learning Rate: 0.0001
Model checkpoint saved!
Epoch duration: 124.32 seconds



Epoch 2/30 - Training: 100%|██████████| 65/65 [01:47<00:00,  1.65s/it, loss=22.2]


...

Epoch [29/30] Validation Loss: 2.4908, Accuracy: 70.96%
Current Learning Rate: 5e-05
Epoch duration: 124.74 seconds



Epoch 30/30 - Training: 100%|██████████| 65/65 [01:48<00:00,  1.67s/it, loss=24.5]



Epoch [30/30] Training Loss: 0.0238, Accuracy: 99.66%


Epoch 30/30 - Validation: 100%|██████████| 17/17 [00:17<00:00,  1.02s/it, loss=0.839]

Epoch [30/30] Validation Loss: 2.3659, Accuracy: 71.54%
Current Learning Rate: 5e-05
Model checkpoint saved!
Epoch duration: 126.10 seconds

